In [ ]:
# Install and Import Libraries
!pip install eemont

import ee
import geemap
import geemap.colormaps as cm
import eemont


In [ ]:
# Authenticate to Earth Engine.
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project =  'ee-nouribrahim25')

**Define AOI**

In [ ]:
coors = [[29.822395, 30.608354],
    [29.822395, 30.754805],
    [30.098427, 30.754805],
    [30.098427, 30.608354],
    [29.822395, 30.608354]]

aoi = ee.Geometry.Polygon(coors)


In [ ]:
# Create an Interactive Map
Map = geemap.Map(width = 800)
Map.add_basemap('HYBRID')
Map.centerObject(aoi , 11)
Map.add_layer(aoi , {} , 'AOI')
Map

**Sentinel-2 Median Composite**

In [ ]:
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(aoi)
      .filterDate("2024","2025")
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
      .maskClouds()
      .scaleAndOffset()
      .spectralIndices(['NDVI', 'NDBI', 'NDWI'])
      .median()
      .clip(aoi)
      )
s2

In [ ]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'NDVI', 'NDBI', 'NDWI']
label = 'landcover'

image = s2.select(bands)

image

In [ ]:
vis_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

Map.add_layer(s2, vis_params, 'Sentinel-2 RGB')
Map

**Indices visualization parameters**

In [ ]:
# NDVI (Normalized Difference Vegetation Index)
ndvi_vis = {
    'min': -0.2,
    'max': 0.8,
    'palette': cm.palettes.ndvi
}

# NDWI (Normalized Difference Water Index)
ndwi_vis = {
    'min': -0.5,
    'max': 0.3,
    'palette': cm.palettes.ndwi
}

# NDBI (Normalized Difference Built-up Index)
ndbi_vis = {
    'min': -0.3,
    'max': 0.2,
    'palette': cm.get_palette('coolwarm')
}

In [ ]:
# visualize spectral indices
Map.addLayer(image.select('NDVI'), ndvi_vis, 'NDVI', False)
Map.addLayer(image.select('NDWI'), ndwi_vis, 'NDWI', False)
Map.addLayer(image.select('NDBI'), ndbi_vis, 'NDBI', False)

Map

**Get the geometries drawn by the user**

In [ ]:
water = Map.user_rois
water.getInfo()

In [ ]:
vegetation = Map.user_rois
vegetation.getInfo()

In [ ]:
urban = Map.user_rois
urban.getInfo()

In [ ]:
bare_soil = Map.user_rois
bare_soil.getInfo()

In [ ]:
# combine all training samples
training_samples = water.merge(vegetation).merge(urban).merge(bare_soil)
training_samples_gdf = geemap.ee_to_gdf(training_samples)

training_samples_gdf.head(12)

In [ ]:
training_samples_gdf.to_file('training_samples.shp')


**Export the Shapefile to Google Drive**

In [ ]:
shp_task = ee.batch.Export.table.toDrive(
    collection= training_samples,
    description='Training_Samples_SHP_4_Classes',
    folder='GEE_Exports',          # Data will be saved in this folder on your Drive
    fileNamePrefix='s2_training_samples_shp',
    fileFormat='SHP'               # This specifies the Shapefile format
)

shp_task.start()

**Exporting Labeled Samples as CSV**

In [ ]:
csv_task = ee.batch.Export.table.toDrive(
    collection=training_samples,
    description='Training_Samples_CSV',
    folder='GEE_Exports',          # Data will be saved in this folder on your Drive
    fileNamePrefix='trining_samples_csv',
    fileFormat='CSV'               # This specifies the CSV format
)

csv_task.start()